In [1]:
###############################################################################################################
##################################### COSMIC #############################################################
###############################################################################################################

import pandas as pd
from pymongo import MongoClient
import pprint
import json

# Connect to MongoDB
client = MongoClient()
db = client.progenetix
bs = db.biosamples

## Problems
# mutant_export_consensus = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicMutantExportCensus.tsv', sep = '\t', low_memory = False)
# mutant_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicMutantExport.tsv', sep = '\t', low_memory = False)
# complete_target_screens = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicCompleteTargetedScreensMutantExport.tsv', sep = '\t', low_memory = False)

# Load data frames
resistance_mutations = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicResistanceMutations.tsv', sep = '\t', low_memory = False)
ncv = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicNCV.tsv', sep = '\t', low_memory = False)
genome_screens = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicGenomeScreensMutantExport.tsv', sep = '\t', low_memory = False)
fusion_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicFusionExport.tsv', sep = '\t', low_memory = False)
breakpoints_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicBreakpointsExport.tsv', sep = '\t', low_memory = False)
struct_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicStructExport.tsv', sep = '\t', low_memory = False)

# Select Pubmed IDs
resistance_mutations = resistance_mutations['Pubmed Id']
ncv = ncv['PUBMED_PMID']
genome_screens = genome_screens['Pubmed_PMID']
fusion_export = fusion_export['PUBMED_PMID']
breakpoints_export = breakpoints_export['Pubmed_PMID']
struct_export = struct_export['PUBMED_PMID']

cosmic = [resistance_mutations, ncv, genome_screens,
          fusion_export, breakpoints_export, struct_export]

pmed_list = []

for i in cosmic:
    for j in i:
        pmed_list.append(j)
        
pmeds = [int(x) for x in pmed_list if str(x) != 'nan']
pmeds = set(pmeds)

db_matches = []
for ids in pmeds:
    hits = db.biosamples.find({"external_references.id":{'$regex': str(ids)},"biosample_status.id":"EFO:0009656"})
    for entry in hits:
        db_matches.append(entry)
        
ex_ref = []

for i in db_matches:
    ex_ref.append(i['external_references'])

pm = []
for i in ex_ref:
    for j in i:
        if 'PMID' in j['id'] and j not in pm:
            pm.append(j)
            
#pprint.pprint(pm)

with open(f"../temp/cosmic_pubmed_ids.json", mode="w") as f:
    f.write(json.dumps(pm))

In [2]:
print(len(pm), len(pmeds), len(pm)/len(pmeds))
# 98 9410 0.010414452709883103

98 9410 0.010414452709883103
